In [6]:
import requests
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

# === Custom Embedding Class for Ollama ===
class OllamaEmbeddings:
    def __init__(self, model="mxbai-embed-large", url="http://localhost:11434/v1/embeddings"):
        self.model = model
        self.url = url

    def embed_documents(self, texts):
        embeddings = []
        for text in texts:
            payload = {"model": self.model, "input": text}
            response = requests.post(self.url, json=payload)
            response.raise_for_status()
            data = response.json()
            embeddings.append(data["data"][0]["embedding"])
        return embeddings

    def embed_query(self, text):
        return self.embed_documents([text])[0]

# === Step 1: User Query ===
user_query = "history of transportation"
print(f"📨 User Query: {user_query}\n")

# === Step 2: Convert query to embedding ===
embedding_model = OllamaEmbeddings()
query_embedding = embedding_model.embed_query(user_query)
print(f"📐 Query Embedding Length: {len(query_embedding)}\n")

# === Step 3: Load Vector Store ===
persist_dir = "./chroma_db"
vectorstore = Chroma(
    persist_directory=persist_dir,
    embedding_function=embedding_model
)

# === Step 4: Retrieve similar chunks ===
semantic_results = vectorstore.similarity_search(user_query, k=5)
print(f"📦 Retrieved {len(semantic_results)} relevant chunks from vector DB.\n")

# === Step 5: Show retrieved chunks ===
for i, doc in enumerate(semantic_results):
    print(f"--- Chunk {i + 1} ---")
    print(doc.page_content)
    print()

# === Step 6: Send retrieved context to LLM (mocked example) ===
context = "\n".join([doc.page_content for doc in semantic_results])

# Optional: Use Ollama to call LLM
llm_payload = {
    "model": "llama3.2-vision:11b",  # change as needed
    "prompt": f"Answer the question based on the following context:\n\n{context}\n\nQuestion: {user_query}\n\nAnswer:",
    "stream": False
}

llm_response = requests.post("http://localhost:11434/v1/completions", json=llm_payload)
llm_response.raise_for_status()
answer = llm_response.json()['choices'][0]['text']

print("🧠 Final Answer from LLM:\n")
print(answer.strip())

📨 User Query: history of transportation

📐 Query Embedding Length: 1024

📦 Retrieved 5 relevant chunks from vector DB.

--- Chunk 1 ---
In ancient times, transportation was limited to walking and animal domestication. Soon four once enter discuss. Draw model adult stand five modern there. No air run win. Sometimes drive manage management. Land commercial guess push. Phone director accept teach short water.

During the Industrial Revolution, steam engines transformed both land and sea travel. Form nor affect some various watch choice cause. Cost home hard general nor administration open. Happen show garden.

--- Chunk 2 ---
The 20th century saw the rise of automobiles and airplanes. Quite theory quite shoulder explain. Education who operation respond better. Think whose whether. Police business language out myself hear.

In ancient times, transportation was limited to walking and animal domestication. Trip material add southern discuss. Information get shoulder better usually. Often vot